In [3]:
from bs4 import BeautifulSoup
import time
from time import sleep
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from random import randint
from selenium.webdriver.chrome.options import Options
import logging
from selenium.common.exceptions import NoSuchElementException

# Configure logging
logging.basicConfig(filename='SystemOut.log',format='%(asctime)s %(message)s',
                   encoding='UTF-8',level=logging.WARNING)

start = time.time()
Data_analyst=[]
info={}
counter=0


#initialization
chrome_options = Options()
chrome_options.add_argument('--headless')  # Optional: Run Chrome in headless mode
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36')
driver = webdriver.Chrome(options=chrome_options)


def get_url():
    position=input("What position are you looking for? ")
    location=input("What is the primary location of this position")
    position_converted=position.lower().replace(' ','+')
    location_converted=location.lower().replace(' ','+')
    url=f'https://ma.indeed.com/jobs?q={position_converted}&l={location_converted}'
    return url

url=get_url()
driver.get(url)
sleep(randint(2, 6))


# Getting the number of pages that will be displayed 
elements=driver.find_elements(By.CSS_SELECTOR,'.css-1g90gv6.eu4oa1w0')
nbr_pages = [content.strip() for element in elements for content in element.text.split('\n')]
print("The number of pages that will be displayed is  : ",len(nbr_pages))

for page in range(0,len(nbr_pages)*10,10): 
    # print(f'{url}+&start={page}')
    driver.get(f'{url}+&start={page}')
    sleep(randint(4, 7))
    
    #Eliminate windows that may appear
    try:
        close_button = driver.find_element(By.CSS_SELECTOR,".css-yi9ndv.e8ju0x51")
        close_button.click()
        print("Button clicked successfully.")
    except :
        print("Button not found. Continuing without clicking.")
       
    
    #Getting the number of positions in each page 
    positions_found=driver.find_elements(By.CSS_SELECTOR,'.jcs-JobTitle.css-jspxzf.eu4oa1w0')   
    print(f"The page number {page//10} contains {len(positions_found)} positions",end='\n')

    #Get the cards containing informations 
    cards=driver.find_elements(By.CSS_SELECTOR,".css-5lfssm.eu4oa1w0")
    for i in cards : 
        def get_info(keyword):
            try : 
                if keyword == 'Position':
                    return i.find_element(By.CSS_SELECTOR,'.jcs-JobTitle.css-jspxzf.eu4oa1w0').text
                elif keyword == 'Company':
                    return i.find_element(By.CSS_SELECTOR,'.company_location.css-12lvszk.e37uo190').text.split("\n")[0]
                elif keyword == 'Location':
                    return i.find_element(By.CSS_SELECTOR,'.company_location.css-12lvszk.e37uo190').text.split("\n")[1]
                elif keyword == 'contract':
                    return i.find_element(By.CSS_SELECTOR,'.css-1ihavw2.eu4oa1w0').text
                elif keyword == 'Description':
                    return i.find_element(By.CSS_SELECTOR,'.job-snippet').text  
                elif keyword == 'url':
                    return i.find_element(By.CSS_SELECTOR,"a").get_attribute("href")
            except NoSuchElementException as err:
                logging.warning(f"Element not found for '{keyword}'")
                return "Empty"
            except IndexError as err:
                logging.warning(f"Location not found for Company/Location ")
                return "Empty"
            
        counter+=1
        info={}
        info ={
            'Position' :get_info('Position'),
            'Company' : get_info('Company'),
            'Location' :get_info('Location'),
            'contract' :get_info('contract'),
            'Description' :get_info('Description'),
            'url':get_info('url')   
        }
        
        if get_info('Position') == "Empty" :
            continue
        Data_analyst.append(info)

# Dataframe creation
df=pd.DataFrame(Data_analyst)
df.to_csv("Indeed.csv",index=False)

print("Number of job offers : " ,len(Data_analyst), "/",counter)
driver.quit() # Closing the browser we opened
end = time.time()

print(end - start,'seconds to complete action!')
print('-----------------------')

What position are you looking for?  Data analyst
What is the primary location of this position Casablanca


The number of pages that will be displayed is  :  5
Button clicked successfully.
The page number 0 contains 15 positions
Button not found. Continuing without clicking.
The page number 1 contains 15 positions
Button not found. Continuing without clicking.
The page number 2 contains 15 positions
Button not found. Continuing without clicking.
The page number 3 contains 15 positions
Button not found. Continuing without clicking.
The page number 4 contains 15 positions
Number of job offers :  75 / 90
99.62131881713867 seconds to complete action!
-----------------------


In [4]:
df.head()

,Position,Company,Location,contract,Description,url
0,Data Analyst Junior - Casablanca - 2023 H/F,MAZARS,Casablanca,CDI,Développement de solutions innovantes sur des ...,https://ma.indeed.com/rc/clk?jk=8aae54b3e2c0be...
1,Data analyst / bi,Distrib-Company,Casablanca,Empty,Nous sommes à la recherche d’un profil « Data ...,https://ma.indeed.com/rc/clk?jk=562f39e2af6f47...
2,Data Analyst - H/F,Bénin Digital,Casablanca,Temps plein,Il devra identifier les écarts et dysfonctionn...,https://ma.indeed.com/rc/clk?jk=071261c087b734...
3,Data Analyst,Novaxys,Casablanca,CDI,Novaxys recrute un Data Analyst pour le compte...,https://ma.indeed.com/rc/clk?jk=a8884c65834ba8...
4,Data Analyst & Project Coordinator,CRIT MAROC,Casablanca,Temps plein\n+1,Nous recherchons un Data Analyst & Project Coo...,https://ma.indeed.com/rc/clk?jk=916ac593fc8180...


In [5]:
with open("SystemOut.log", 'r') as file:
    print(file.readline())

2024-01-07 13:08:28,138 Message: no such element: Unable to locate element: {"method":"css selector","selector":".css-1ihavw2.eu4oa1w0"}

